In [3]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import r2_score
from datetime import date
def dateFromFloat(date_Float:float):
    day= date_Float % 100
    month = (date_Float // 100)%100 
    year = date_Float // 10000
    return date(year,month,day)
    
def daysFromPredict(predictRaceDate_date:date, raceDate_int:float):    
    raceDate = dateFromFloat(raceDate_int)
    return abs(predictRaceDate_date - raceDate).days
def lambda_daysFromPredict(predictRaceDate_date,row): 
    return daysFromPredict(predictRaceDate_date,row['rateDate'])
def lambda_weightedPlace(totaldays:float,row): 
    return row['place']*(1-row['days']/totaldays)
  
    
predictRaceDate =20181205
predictRaceDate_date = dateFromFloat(predictRaceDate)
seqOfRaceDay = 9
startOfSeason=20180901
#Load the data
file0 = "/Users/apple/Workspace/intelliJ/jockey/"+str(predictRaceDate)+"_"+str(seqOfRaceDay)+"_history.csv"
file1 = "/Users/apple/Workspace/intelliJ/jockey/"+str(predictRaceDate)+"_"+str(seqOfRaceDay)+"_predict.csv"
dataset0 = pd.read_csv(file0)
dataset1 = pd.read_csv(file1)


In [5]:
allHorse = dataset0['horseId'].unique() # all horseId
#allHorse =['V088']
allPredict_FinishTime=[]
allPredict_Place=[]
 # We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['raceClass', 'distance','draw','addedWeight','declaredHorseWeight']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['course','jockey']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
for idx, horseId in enumerate(allHorse):
    print(horseId)
    if(horseId=='B111'):
        allPredict_FinishTime.append(1000)  # will print a message to the console
        allPredict_Place.append(1000)
        continue
    train_dataset=dataset0[dataset0['horseId']==horseId]
    test_dataset=dataset1[dataset1['horseId']==horseId]
    #latest place 
    X_place = train_dataset[['rateDate','place']]
    X_place =X_place[X_place['rateDate']>=startOfSeason]
    
    X_place['days']=X_place.apply (lambda row: lambda_daysFromPredict (predictRaceDate_date, row),axis=1)
   
    totalDays=X_place.days.sum()
    X_place['weightedPlace']=X_place.apply (lambda row: lambda_weightedPlace (totalDays,row),axis=1)
    #print(X_place)
    predictedPlace = X_place.weightedPlace.sum()
   
    
    #regression model of lbw or finishTime
    X=train_dataset[['raceClass','distance','course','draw','addedWeight','declaredHorseWeight','jockey']]
    y=train_dataset[['finishTime']]

    
    X_test=test_dataset[['raceClass','distance','course','draw','addedWeight','declaredHorseWeight','jockey']]

    X=pd.concat([X,X_test])

    #y=dataset0.iloc[:,12].values #finishTime
   
   


    X_arr=preprocessor.fit_transform(X)
    
    #dummy value trap
    dummyVariableTrap=[]
    offset = len(numeric_features) 
    for i, val in enumerate(categorical_features):
        dummyVariableTrap.append(offset + X[val].nunique() -1)
        offset = offset + X[val].nunique()
   
    X_arr=np.delete(X_arr,dummyVariableTrap,1)
    

    #X_new=np.append(arr=np.ones((np.size(X_arr,0),1),dtype=int),values=X_arr,axis=1)
    #feature = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    #X_opt = X_new[:,feature]
    #X_train=X_opt[:-1,:]
    #regresssor_OLS=sm.OLS(endog=y,exog=X_train).fit()
    #regresssor_OLS.summary()

    if(X_opt) :
        del X_opt
    X_opt = X_arr
   
    X_train=X_opt[:-1,:]
    reg = LinearRegression()
    reg.fit(X_train,y)
    y_train=reg.predict(X_train)
    
    score =r2_score(y, y_train) 
    
    X_predict=np.array([X_opt[-1,:]])
    y_predict= reg.predict(X_predict)
    allPredict_FinishTime.append(y_predict[0][0])  # will print a message to the console
    allPredict_Place.append(predictedPlace)

df = pd.DataFrame(data=[allHorse,allPredict_FinishTime,allPredict_Place]).transpose()
df.columns=['horseId','finishTime','place']
df = df.sort_values(by=['finishTime'])
print(df)
df = df.sort_values(by=['place'])
print(df)

B210


NameError: name 'X_opt' is not defined